In [1]:
#112356012 資碩一 余采嬙
#引用相關套件
import struct
import socket
from threading import Thread
#The multi-thread mechanism is used to implement the upload and download functions of the TFTP Server at the same time.

In [2]:
# Client download thread
def download_thread(fileName, clientInfo):
    print("Responsible for processing client download files")
    
    # Create a UDP socket
    s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM) 
    
    fileNum = 0 #Indicates the serial number of the received file
    
    try:
        f = open(fileName,'rb') #n binary read mode ('rb') ???
    except:
        # Packing
        # !: Indicates that we want to use network character order parsing because our data is received from the network. 
        #    When transmitting on the network, it is the network character order. 
        # H: The following H indicates the id of an unsigned short.
        # b: signed char
        errorData = struct.pack('!HHHb', 5, 5, 5, fileNum) 
        
        # Send an error message
        s.sendto(errorData, clientInfo)   #Sent the message when the file does not exist
        
        exit()  #Exit the download thread #???
        
    while True:
        # Read file contents 512 bytes from local server
        readFileData = f.read(512) #???
        
        # The block number starts at 0 and increments by one each time. Its range is [0, 65535]
        #roll-over 
        if fileNum == 65535:
          fileNum = 0 #wrap to zero #block counter going back to 0 after 65535, which gives an essentially unlimited transfer file size.
        fileNum += 1 

        # Packing
        # !: Indicates that we want to use network character order parsing because our data is received from the network. 
        #    When transmitting on the network, it is the network character order.
        # First H: 3(Data)
        # Seond H: Block number
        sendData = struct.pack('!HH', 3, fileNum) + readFileData #opcode(3) # blockNum #Data  

        # Send file data to the client
        s.sendto(sendData, clientInfo)  #Data sent for the first time 
        
        # When the data received by the client is less than 516 bytes, it means that the transmission is completed!
        if len(sendData) < 516: #??? 
            print("User"+str(clientInfo), end='')
            print('：Download '+fileName+' completed！')
            break
            
        # Receiving data for the second time
        recvData, clientInfo = s.recvfrom(1024) #???
        #print(recvData, clientInfo)

        # Unpacking
        packetOpt = struct.unpack("!H", recvData[:2])  #Opcode #:2
        packetNum = struct.unpack("!H", recvData[2:4]) #Block number 
        
        #print(packetOpt, packetNum)
        
        if packetOpt[0] != 4 or packetNum[0] != fileNum: #
            print("File transfer error！")
            break #???
            
    # Close file
    f.close()
    
    # Close UDP port
    s.close()

    # Exit the download thread
    exit() #must???

In [3]:
# Client uploading thread
def upload_thread(fileName, clientInfo):
    print("Responsible for processing client upload files")
    
    fileNum = 0 #Indicates the serial number of the received file
    
    # Open the file in binary mode
    f = open(fileName, 'wb')
    
    # Create a UDP port
    s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    
    # Packing 
    # struct.pack(fmt, v1, v2, ...): Encapsulate data into strings according to the given format(fmt) 
    # !: Indicates that we want to use network character order parsing because our data is received from the network. 
    #    When transmitting on the network, it is the network character order. 
    # H: The following H indicates the id of an unsigned short.
    # unsign short:16bits
    sendDataFirst = struct.pack("!HH", 4, fileNum) 

    # Reply to the client upload request
    s.sendto(sendDataFirst, clientInfo)  #Sent with a random port at first time

    while True:
        # Receive data sent by the client
        recvData, clientInfo = s.recvfrom(1024) #Client connects to my random port at second time
        
        #print(recvData, clientInfo)
        
        # Unpacking
        packetOpt = struct.unpack("!H", recvData[:2])  #Identify opcode
        packetNum = struct.unpack("!H", recvData[2:4]) #Block number
        
        #print(packetOpt, packetNum)
        
        # Client upload data
        # opcode == 3 means Data
        if packetOpt[0] == 3 and packetNum[0] == fileNum:
            #　Save data to file
            f.write(recvData[4:])
            
            # Packing
            sendData = struct.pack("!HH", 4, fileNum)
            
            # Reply client's ACK signal
            s.sendto(sendData, clientInfo) #The second time using a random port to sent
            
            
            if fileNum == 65535:
                fileNum = 0 #wrap to zero #block counter going back to 0 after 65535, which gives an essentially unlimited transfer file size.
            fileNum += 1    
            
            #If len(recvData) < 516 means the file goes to the end
            if len(recvData) < 516:
                print("User"+str(clientInfo), end='')
                print('：Upload '+fileName+' complete!')
                break
                
    # Close the file
    f.close()
    
    # Close UDP Port
    s.close()
    
    # Exit upload thread
    exit()

In [4]:
# Main function
def main():
    # Create a UDP port & use IP Protocol
    s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    
    # Resolve duplicate binding ports
    # setsockopt(level,optname,value)
    # Level: defines which option will be used. Usually use "SOL_SOCKET", it means the socket option being used.
    # optname: Provide special options for use. Ex: SO_BINDTODEVICE, SO_BROADCAST, SO_DONTROUTE, SO_REUSEADDR, etc.
    s.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1) 
    
    # Bind local host and port number 6969
    s.bind(('127.0.0.1', 6969))
    
    print("TFTP Server start successfully!")
    print("Server is running...")
    
    while True:
        
        # Receive messages sent by the client
        recvData, clientInfo = s.recvfrom(1024)  #　Client connects to port 6969 at the first time
        #print(clientInfo) 
        
        # Unpacking
        # !: Indicates that we want to use network character order parsing because our data is received from the network. 
        #    When transmitting on the network, it is the network character order. 
        # b: signed char
        # There can be one number before each format, indicating the number
        # s: char[]
        if struct.unpack('!b5sb', recvData[-7:]) == (0, b'octet', 0): #octet(Mode)
            opcode = struct.unpack('!H',recvData[:2])  #　Opcode
            fileName = recvData[2:-7].decode('ascii') #　Filename
            #Opcode(2bytes)#Filename(String)#0(1 byte)#Mode(string)#0(1 byte)

            # Request download
            # opcode == 1 means download
            if opcode[0] == 1:
                t = Thread(target=download_thread, args=(fileName, clientInfo)) 
                t.start() # Start the download thread
                
            # Request uploading
            # opcode == 2 means uploading
            elif opcode[0] == 2:
                t = Thread(target=upload_thread, args=(fileName, clientInfo)) 
                t.start() # Start uploading thread
                
    # Close UDP port
    s.close()

In [5]:
# Call the main function
if __name__ == '__main__':
    with open('bigfile.txt', 'w') as file:
        file.write('test123')
    main()

TFTP Server start successfully!
Server is running...
Responsible for processing client download files
User('127.0.0.1', 60636)：Download bigfile.txt completed！


KeyboardInterrupt: 

: 